In [ ]:
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

img_dir = './dataset/train'
categories = ['Bird', 'Dog']
np_classes = len(categories)

image_w = 64
image_h = 64


pixel = image_h * image_w * 3

X_train = []
Y_train = []

for idx, bird in enumerate(categories):
    img_dir_detail = img_dir + "/" + bird
    files = glob.glob(img_dir_detail+"/*.jpg")

 
    for i, f in enumerate(files):
        try:
            img = Image.open(f)
            img = img.convert("RGB")
            img = img.resize((image_w, image_h))
            data = np.asarray(img)
            #Y는 0 아니면 1이니까 idx값으로 넣는다.
            X_train.append(data)
            Y_train.append(idx)
            if i % 3 == 0:
                print(bird, " : ", f)
        except:
            print(bird, str(i)+" 번째에서 에러 ")
X_train = np.array(X_train)
Y_train = np.array(Y_train)


#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

#xy = (X_train, X_test, Y_train, Y_test)
#np.save("./dataset/binary_image_data.npy", xy)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

#X_train, X_test, Y_train, Y_test = np.load('./dataset/binary_image_data.npy')
print(X_train.shape)
print(X_train.shape[0])
print(np.bincount(Y_train))
#print(np.bincount(Y_test))

image_w = 64
image_h = 64
X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255


with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_dir = './model'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + "/dog_bird_classify.model"
    
    checkpoint = ModelCheckpoint(filepath=model_path, monitor='loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='loss', patience=7)



model.summary()

history = model.fit(X_train, Y_train, batch_size=64, epochs=2000, callbacks=[checkpoint, early_stopping])

print("train 정확도 : %.2f " %(model.evaluate(X_train, Y_train)[1]))

plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'acc'], loc='lower left')
plt.show()

from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

import tensorflow as tf

seed = 5
tf.set_random_seed(seed)
np.random.seed(seed)

caltech_dir = './dataset/val'


image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
Y = []
filenames = []
files = glob.glob(caltech_dir+"/*/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)

    filenames.append(f)
    X.append(data)

X = np.array(X)
X = X.astype(float) / 255
model = load_model('./model/dog_bird_classify.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0
count = 0
acc = 0

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def img_process(filename):
    image=mpimg.imread(filename)
   
    plt.figure()
    plt.imshow(image)

for i in prediction:
    if i >= 0.5: 
        img_process(caltech_dir+'/'+filenames[cnt].split("\\")[1]+'/'+filenames[cnt].split("\\")[2])
        plt.title([filenames[cnt].split("\\")[2] + '  is dog'])
        if filenames[count].split("\\")[1] == "dog": acc += 1
    else : 
        img_process(caltech_dir+'/'+filenames[cnt].split("\\")[1]+'/'+filenames[cnt].split("\\")[2])
        plt.title([filenames[cnt].split("\\")[2] + '  is bird'])
        if filenames[count].split("\\")[1] == "bird": acc += 1
    cnt += 1
    count += 1
    
print("test 정확도 : ", acc/20)
